<a href="https://colab.research.google.com/github/ysMarcos/projeto-python/blob/main/projeto_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports das Bibliotecas usadas

In [2]:
#Imports
import requests as req
import hashlib
import pandas as pd
import sqlite3
import json
from google.colab import userdata

# Funções utilizadas

In [3]:
#Funcao responsavel para encryptografar para md5 de acordo com a documentacao da api da Marvel
def hashToMD5(publicKey, privateKey):
  #Valor 1 eh o timestamp de exemplo usado na documentacao
  return hashlib.md5(str.encode(str(1) + str(privateKey) + str(publicKey)))

In [4]:
# Funcao responsavel pelas requisicoes
def get(url: str, endpoint: str, headers: dict, params: dict):
  try:
    response = req.get(
        url = url+endpoint,
        headers = headers,
        params = params
      )
    return response.json()
  except Exception as e:
    print(e)

In [23]:
#Funcao responsavel por mapear os dados de acordo com o input recebido
#Este mapeador eh generico e limitado em relacao a tipagens, para a construcao de um
#banco de dados mais complexo e corretamente tipado, seria bom uma classe para os endpoints
#que eh implementada os metodos de ETL
def mapper(keys, values):
  arr = []
  obj = {}
  for i in range(len(values)):
    for j in range(len(keys)):
      #Converte Listas e Dicionarios em string
      if isinstance(values[i][keys[j]], (list, dict)):
        obj[keys[j]] = json.dumps(values[i][keys[j]])
      else:
        obj[keys[j]] = values[i][keys[j]]
    arr.append(obj)
    obj = {}
  return arr

In [33]:
#Funcoes responsaveis pela transformacao dos dados em um Dataframe e salvar o objeto passado
def saveToCSV(obj, csvPath):
  df = pd.DataFrame(obj)
  df.to_csv(csvPath + '.csv', encoding = 'utf-8', index = False, header = True)

def saveToSqlite(obj, tableName, dbPath='bd.db', uniqueColumns = None):
  try:
    df = pd.DataFrame(obj) if not isinstance(obj, pd.DataFrame) else obj
    with sqlite3.connect(dbPath) as con:
      df.to_sql(tableName, con, if_exists='append', index=False)

      if(uniqueColumns):
        uniqueClause = ", ".join(uniqueColumns)
        query = f"""
          DELETE FROM {tableName}
          WHERE rowid NOT IN (
              SELECT MIN(rowid)
              FROM {tableName}
              GROUP BY {uniqueClause}
          );
          """
        con.execute(query)
  except Exception as e:
    print(f"Erro: {e}")

# Endpoints

In [31]:
#Variaveis que serao utilizadas em todos os endpoints da marvel
url = 'http://gateway.marvel.com/v1/public/'
publicKey = userdata.get('publicKey')
privateKey = userdata.get('privateKey')
securityHash = hashToMD5(publicKey, privateKey).hexdigest()
dbPath = userdata.get('dbPath')

headers = {
    'Accept': "*/*"
}

params = {
    "ts": 1,
    "apikey": publicKey,
    "hash": securityHash,
    "limit": 100
}

## Characters

In [37]:
endpoint = 'characters'

result = get(url, endpoint, headers, params)['data']['results']
characters = mapper(['id', 'name'], result)
saveToCSV(csvPath = endpoint, obj = characters)
saveToSqlite(characters, endpoint, dbPath, ['id'])

## Comics

In [35]:
endpoint = 'comics'

result = get(url, endpoint, headers, params)['data']['results']
comics = mapper(['id', 'digitalId', 'title', 'issueNumber', 'dates', 'prices', 'resourceURI'], result)
saveToCSV(csvPath = endpoint, obj = comics)
saveToSqlite(comics, endpoint, dbPath, ['id'])

## Events

In [36]:
endpoint = 'events'

result = get(url, endpoint, headers, params)['data']['results']
events = mapper(['id', 'title', 'description', 'resourceURI', 'start', 'end'], result)
saveToCSV(csvPath = endpoint, obj = events)
saveToSqlite(events, endpoint, dbPath, ['id'])